In [2]:
import datetime
from datetime import timedelta
# 用于处理中国节假日和調休的库，需要事先安装: pip install chinese_calendar
from chinese_calendar import is_workday

class TenderTimelineCalculator:
    """
    招标时间节点计算器类。
    封装了一系列根据初始时间和业务逻辑推算后续关键节点的方法。
    """

    @staticmethod
    def get_next_workday(target_date: datetime.date) -> datetime.date:
        """
        工具方法：获取下一个工作日。
        如果传入的 target_date 本身是工作日，则直接返回。
        如果传入的 target_date 是休息日（周末或法定节假日），这往后推算直到找到第一个工作日。
        依赖 chinese_calendar 库来准确判断中国的节假日和调休情况。
        """
        current_check_date = target_date
        # 循环检查，只要当前日期不是工作日，就加一天继续检查
        while not is_workday(current_check_date):
            current_check_date += timedelta(days=1)
        return current_check_date

    def calculate(self, start_date_str: str) -> dict:
        """
        核心计算方法。
        接收字符串格式的发布时间，返回包含所有计算节点的字典。
        """
        # 1. 解析输入时间：招标公告日即为第一日 (T)
        try:
            t1_announcement_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d").date()
        except ValueError:
            return {"error": "日期格式错误，请使用 YYYY-MM-DD 格式 (例如 2023-10-27)"}

        results = {}
        results["T1_招标公告发布日"] = t1_announcement_date

        # 2. 招标文件发售开始日：自第二日开始 (T+1)
        # 逻辑：公告日是第1天，发售开始是第2天。
        t2_sale_start_date = t1_announcement_date + timedelta(days=1)
        results["T2_发售开始日"] = t2_sale_start_date

        # 3. 招标文件发售截止时间：持续五日，遇休息日顺延
        # 逻辑：从发售开始日算起，持续5天 (即开始日再加4天)。
        # 规则：截至时间如果在休息日则顺延到下一个工作日。
        t2_sale_end_initial = t2_sale_start_date + timedelta(days=4)
        t2_sale_end_final = self.get_next_workday(t2_sale_end_initial)
        results["T2_发售截止日(已遇休顺延)"] = t2_sale_end_final

        # 4. 投标人提答疑截止日：截至后一天，遇休息日顺延
        # 逻辑：发售截止日的第二天。
        # 规则：如果为休息日则顺延到下一个工作日。
        t3_qa_deadline_initial = t2_sale_end_final + timedelta(days=1)
        t3_qa_deadline_final = self.get_next_workday(t3_qa_deadline_initial)
        results["T3_提答疑截止日(已遇休顺延)"] = t3_qa_deadline_final

        # 5. 开评标日：提答疑后第十六日，遇休息日顺延
        # 逻辑：答疑截止日之后的第16天 (即答疑截止日 + 16天)。
        # 规则：如果在休息日则顺延到下一个工作日。
        t4_bid_opening_initial = t3_qa_deadline_final + timedelta(days=16)
        t4_bid_opening_final = self.get_next_workday(t4_bid_opening_initial)
        results["T4_开评标日(已遇休顺延)"] = t4_bid_opening_final

        # 6. 中标公示期：开评标后三日 (日历天)
        # 逻辑：开标日后的第一天开始，持续3天。通常公示期包含节假日。
        t5_notice_start = t4_bid_opening_final + timedelta(days=0)
        t5_notice_end = t4_bid_opening_final + timedelta(days=3)
        results["T5_中标公示开始日"] = t5_notice_start
        results["T5_中标公示结束日"] = t5_notice_end

        # 7. 中标通知发出日：第四日，遇休息日顺延
        # 逻辑：开评标后的第四日 (也就是公示结束后的第一天)。
        # 规则：如果在休息日则顺延到下一个工作日。
        t6_issuance_initial = t4_bid_opening_final + timedelta(days=4)
        t6_issuance_final = self.get_next_workday(t6_issuance_initial)
        results["T6_中标通知书发出日(已遇休顺延)"] = t6_issuance_final

        return results

# ==================
# 主程序入口示例
# ==================
if __name__ == "__main__":
    # 实例化计算器
    calculator = TenderTimelineCalculator()

    # 模拟输入：假设招标公告发布时间是一个周五，后面紧接着周末
    input_date_str = "2025-11-28"
    print(f"=== 开始计算，基准公告日期: {input_date_str} ===")
    print("注：此计算依赖 chinese_calendar 库以准确识别中国节假日与调休。")
    print("-" * 50)

    # 执行计算
    timeline = calculator.calculate(input_date_str)

    # 格式化输出结果
    if "error" in timeline:
        print(f"错误: {timeline['error']}")
    else:
        # 定义清晰的打印顺序
        print_order = [
            "T1_招标公告发布日",
            "T2_发售开始日",
            "T2_发售截止日(已遇休顺延)",
            "T3_提答疑截止日(已遇休顺延)",
            "T4_开评标日(已遇休顺延)",
            "T5_中标公示开始日",
            "T5_中标公示结束日",
            "T6_中标通知书发出日(已遇休顺延)"
        ]
        for key in print_order:
            date_val = timeline.get(key)
            # 输出格式：描述 : YYYY-MM-DD (星期几)
            weekday_cn = ["一", "二", "三", "四", "五", "六", "日"][date_val.weekday()]
            print(f"{key:<25} : {date_val} (星期{weekday_cn})")

    print("-" * 50)

# 关于代码逻辑结构的展示：
# 1.  **TenderTimelineCalculator 类**：代码的核心容器，负责封装所有相关的逻辑。
# 2.  **get_next_workday 静态方法**：
#     *   输入一个日期。
#     *   利用 `chinese_calendar.is_workday` 判断该日期是否为工作日（处理了周末和法定节假日调休）。
#     *   如果不是工作日，则循环递增日期，直到找到下一个工作日并返回。这是处理所有“遇休息日顺延”规则的基础工具。
# 3.  **calculate 方法**：
#     *   接收用户输入的起始字符串日期并解析。
#     *   严格按照需求描述的七个步骤依此计算日期：
#         *   T1 (公告日): 输入日期本身。
#         *   T2 (发售开始): T1 + 1天。
#         *   T2 (发售截止): T2开始日 + 4天，然后调用 `get_next_workday` 处理顺延。
#         *   T3 (答疑截止): T2截止日 + 1天，然后调用 `get_next_workday` 处理顺延。
#         *   T4 (开标日): T3答疑日 + 16天，然后调用 `get_next_workday` 处理顺延。
#         *   T5 (公示期): T4开标日后接下来的3个日历日（无顺延规则）。
#         *   T6 (通知书发出): T4开标日 + 4天，然后调用 `get_next_workday` 处理顺延。
#     *   最终返回一个包含所有关键时间节点的字典。
# 4.  **主程序入口**：提供了示例调用，演示如何实例化类、输入数据、执行计算并格式化打印结果以便验证。

=== 开始计算，基准公告日期: 2025-11-28 ===
注：此计算依赖 chinese_calendar 库以准确识别中国节假日与调休。
--------------------------------------------------
T1_招标公告发布日                : 2025-11-28 (星期五)
T2_发售开始日                  : 2025-11-29 (星期六)
T2_发售截止日(已遇休顺延)           : 2025-12-03 (星期三)
T3_提答疑截止日(已遇休顺延)          : 2025-12-04 (星期四)
T4_开评标日(已遇休顺延)            : 2025-12-22 (星期一)
T5_中标公示开始日                : 2025-12-22 (星期一)
T5_中标公示结束日                : 2025-12-25 (星期四)
T6_中标通知书发出日(已遇休顺延)        : 2025-12-26 (星期五)
--------------------------------------------------
